In [1]:
import pandas as pd
import numpy as np
import math


In [31]:
#Read the csv.file, storing it to varaible "data_enjoy":

data_enjoy = pd.read_csv("C:/Users/User/Desktop/USC/Machine Learning/dt_data.csv")

#Replace the 'Enjoy' labels:"1" for "yes" and "0" for "no":

(data_enjoy['Enjoy']).replace(['Yes', 'No'], [1, 0], inplace=True)

#Save all attributes to variable fs_of_enjoy to run program with ease:

fs_of_enjoy = ['Occupied', 'Price', 'Music', 'Location', 'VIP', 'Favorite Beer']

print(data_enjoy)

    Occupied      Price  Music       Location  VIP Favorite Beer  Enjoy
0       High  Expensive   Loud        Talpiot   No            No      0
1       High  Expensive   Loud    City-Center  Yes            No      1
2   Moderate     Normal  Quiet    City-Center   No           Yes      1
3   Moderate  Expensive  Quiet  German-Colony   No            No      0
4   Moderate  Expensive  Quiet  German-Colony  Yes           Yes      1
5   Moderate     Normal  Quiet      Ein-Karem   No            No      1
6        Low     Normal  Quiet      Ein-Karem   No            No      0
7   Moderate      Cheap   Loud  Mahane-Yehuda   No            No      1
8       High  Expensive   Loud    City-Center  Yes           Yes      1
9        Low      Cheap  Quiet    City-Center   No            No      0
10  Moderate      Cheap   Loud        Talpiot   No           Yes      0
11       Low      Cheap  Quiet        Talpiot  Yes           Yes      0
12  Moderate  Expensive  Quiet  Mahane-Yehuda   No           Yes

In [3]:
#First, define the entropy function:

def get_entropy(data, label):
    label_value, label_count = np.unique(data[label], return_counts=True)

    entropy_value = np.sum(np.fromiter(((-label_count[i]/np.sum(label_count))*
                                        np.log2(label_count[i]/np.sum(label_count))
                            for i in range(len(label_value))),float))
    return (entropy_value)

#Test the above fucntion with calculating total entropy of night-outs data
get_entropy(data_enjoy,'Enjoy')

0.9760206482366149

In [24]:
#Then,define the function of calculating information gain

def info_gain(data, attribute, label):
    data_entropy = get_entropy(data, label)

    feat_value, feat_number = np.unique(data[attribute], return_counts=True)

#feat_value = the feature values a certain feature (attribute) has,
#feat_number = the numbers of every feature values      
    
    weighted_feat_entropy = np.sum([(feat_number[i]/np.sum(feat_number))*
                            get_entropy((data_enjoy.where(data_enjoy[attribute] == feat_value[i]).dropna()),label)
                            for i in range(len(feat_value))])

    information_gain = data_entropy - weighted_feat_entropy

    return(information_gain)


In [26]:
#Define the function to find the attribute that gives us the largest information gain 

def best_attri(data,attributes,label):
    
    for i in range(len(attributes)):
        gain = info_gain(data,(attributes[i]),label)
        
        best_attri_index = np.argmax(gain)
    
    return (best_attri_index)

In [27]:
#Constructing the decision tree with above functions

def create_tree (data,attributes,label,root_node = None):

#If all entities belongs to the same label class, returning that label class:     
    
    if len(np.unique(data[label])) <= 1: 
        return np.unique(data[label])[0]

#If no attributes remained,returning root node:
    
    elif len(attributes) == 0:
        return root_node

#Elsewhile, keep constructing the decision tree:

    else:

#Update the root node value:
        root_node = np.unique(data[label])[np.argmax(np.unique(data[label],return_counts=True)[1])]

    best_attribute_number = best_attri(data,attributes, label)
#best_attribute_number: the index in attribute list of the attribute having the largest info gain    
       
    best_attribute_name = attributes[best_attribute_number]
#best_attribute_name: that attribue's name 


#Delete the used attributes from the attribute list:
      
    attributes = [k for k in attributes if k!= best_attribute_name] 

#Define the decision tree's foundation:    
    
    Decision_tree = {best_attribute_name:{}}

#Start to build the tree:    
       
    for i in np.unique(data[best_attribute_name]):
        
        i = i 

#Reconstruct the data everytime we find a best-information gain attribute:        
        new_data_df = data_enjoy.where((data[best_attribute_name])==i).dropna()
        
        Decision_tree[best_attribute_name][i] = create_tree(new_data_df,attributes,label,root_node)
    
    return Decision_tree  

In [28]:
create_tree(data_enjoy,fs_of_enjoy,'Enjoy',root_node = None)

{'Occupied': {'High': {'Price': {'Cheap': 1.0,
    'Expensive': {'Music': {'Loud': {'Location': {'City-Center': 1.0,
        'Talpiot': 0.0}}}},
    'Normal': {'Music': {'Loud': 1.0, 'Quiet': 0.0}}}},
  'Low': {'Price': {'Cheap': {'Music': {'Loud': 1.0, 'Quiet': 0.0}},
    'Expensive': 0.0,
    'Normal': {'Music': {'Quiet': {'Location': {'City-Center': {'VIP': {'No': {'Favorite Beer': {'No': 0.0}}}},
        'Ein-Karem': 0.0}}}}}},
  'Moderate': {'Price': {'Cheap': {'Music': {'Loud': {'Location': {'Mahane-Yehuda': 1.0,
        'Talpiot': 0.0}}}},
    'Expensive': {'Music': {'Quiet': {'Location': {'German-Colony': {'VIP': {'No': 0.0,
          'Yes': 1.0}},
        'Mahane-Yehuda': 1.0}}}},
    'Normal': 1.0}}}}

In [29]:
#Build the classifier 

def classifier(my_tree, attrs_to_categorize, test_data):

#Because the data type of decision tree is dictionary,we use index to filter through the decision tree:    
    Dict = my_tree[list(my_tree.keys())[0]]
    Index = attrs_to_categorize.index(list(my_tree.keys())[0])
    class_Label = '0'
    
    for key in Dict.keys():
        if test_data[Index] == key:
            if type(Dict[key]).__name__ == 'dict':
                class_Label = classifier(Dict[key],attrs_to_categorize, test_data)
            else:
                class_Label = Dict[key]
    return class_Label

In [30]:
tree_enjoy = create_tree(data_enjoy,fs_of_enjoy,'Enjoy',root_node = None)

#Entering a case for prediciton:

classifier(tree_enjoy,['Occupied', 'Price', 'Music', 'Location', 'VIP', 'Favorite Beer'],
           ['Moderate','Cheap','Loud','City-Center','No','No'])

'0'